In [66]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0) 

import pandas as pd
import random
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

import os

import numpy as np
import torch.nn.functional as F

from google.colab import drive
drive.mount('/content/drive')

from sklearn.metrics import accuracy_score
from datetime import datetime
from IPython.display import clear_output 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [67]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
#!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.12.1+cu113


In [68]:
%cd /content/drive/MyDrive
label = pd.read_csv("phenotypic_training.csv")
labellist = []
for i in label["DX"]:
    if i == "ADHD":
        labellist.append([0])
    else:
        labellist.append([1])
labellist=torch.tensor(labellist)

/content/drive/MyDrive


In [69]:
train_dataset=[]
graphlist=[]
import networkx as nx
import torch_geometric
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.utils import degree
#from torch.utils.data import DataLoader
from torch_geometric.utils import erdos_renyi_graph, to_networkx, from_networkx
%cd /content/drive/MyDrive/Training
for i in range(1, 600):
  if os.path.isfile("sub-"+ "{0:0>3}".format(i)+"/timeseries_aal.csv"):
    tmp = pd.read_csv("sub-"+ "{0:0>3}".format(i)+"/timeseries_aal.csv", header = None)
    tmp = tmp.T
    tmp = tmp.corr(method="pearson")
    tmp=(tmp.to_numpy())
    tmp=torch.from_numpy(tmp) 
    tmp=torch.div(tmp,tmp.max())
    G = nx.from_numpy_matrix(np.array(tmp))
    centrality = nx.betweenness_centrality(G)
    x = []
    for i in range(len(centrality)):
      x.append([centrality[i]])
    x = np.array(x).reshape(116,1)
    data = from_networkx(G)
    data.x = torch.FloatTensor(x)
    graphlist.append(data)

    #graphlist.append(tmp.type(torch.FloatTensor))

/content/drive/MyDrive/Training


In [70]:
for i in range(len(graphlist)):
  graphlist[i].y = int(labellist[i])

In [71]:
graphlist[0].edge_index

tensor([[  0,   0,   0,  ..., 115, 115, 115],
        [  0,   1,   2,  ..., 113, 114, 115]])

In [72]:
dataset = random.sample(graphlist, len(graphlist))
train_dataset = dataset[:150]
test_dataset = dataset[150:]
train_loader = DataLoader(train_dataset, batch_size=25, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=25, shuffle=True)

In [73]:
from torch_geometric.nn import GATConv, GraphConv, GCNConv
from torch_geometric.nn.conv.gatv2_conv import GATv2Conv
from torch_geometric.nn import global_mean_pool, global_sort_pool

class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()

        self.in_head = 8
        self.out_head = 1
        self.conv1 = GCNConv(in_channels=1, out_channels=16)
        self.conv2 = GCNConv(in_channels=16, out_channels=32)
        self.conv3 = GCNConv(in_channels=32, out_channels=32)
        self.lin1 = torch.nn.Linear(32, 16)
        self.lin2 = torch.nn.Linear(16, 1)
        self.sig = torch.nn.Sigmoid()

    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.weight, data.batch     
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        #print(x.size())
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        #print(x.size())
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        #print(x.size())
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        #x = F.dropout(x, p=0.2, training=self.training)
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        #print(x)
        
        return x
        #return F.log_softmax(x, dim=1)

In [74]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 1000):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6938
Epoch: 002, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6929
Epoch: 003, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6929
Epoch: 004, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6935
Epoch: 005, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6927
Epoch: 006, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6930
Epoch: 007, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6936
Epoch: 008, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6907
Epoch: 009, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6917
Epoch: 010, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6915
Epoch: 011, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6939
Epoch: 012, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6912
Epoch: 013, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6938
Epoch: 014, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6901
Epoch: 015, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6916
Epoch: 016, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6944
Epoch: 0

In [75]:
from torch_geometric.nn import GATConv, GraphConv
from torch_geometric.nn.conv.gatv2_conv import GATv2Conv
from torch_geometric.nn import global_mean_pool, global_sort_pool

class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()

        self.in_head = 8
        self.out_head = 1
        self.conv1 = GraphConv(in_channels=1, out_channels=16)
        self.conv2 = GraphConv(in_channels=16, out_channels=32)
        self.conv3 = GraphConv(in_channels=32, out_channels=32)
        self.lin1 = torch.nn.Linear(32, 16)
        self.lin2 = torch.nn.Linear(16, 1)
        self.sig = torch.nn.Sigmoid()

    def forward(self, data):
        x, edge_index, weight, batch = data.x, data.edge_index, data.weight, data.batch     
        x = self.conv1(x, edge_index, weight)
        x = F.relu(x)
        #print(x.size())
        x = self.conv2(x, edge_index, weight)
        x = F.relu(x)
        #print(x.size())
        x = self.conv3(x, edge_index, weight)
        x = F.relu(x)
        #print(x.size())
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        #x = F.dropout(x, p=0.2, training=self.training)
        x = self.lin1(x)
        x = self.lin2(x)
        #print(x)
        
        return x
        #return F.log_softmax(x, dim=1)

In [76]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 1000):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 21.6387
Epoch: 002, Train Acc: 0.4867, Test Acc: 0.5200, Loss: 1.4543
Epoch: 003, Train Acc: 0.5200, Test Acc: 0.4600, Loss: 1.0046
Epoch: 004, Train Acc: 0.5400, Test Acc: 0.6400, Loss: 0.6124
Epoch: 005, Train Acc: 0.4867, Test Acc: 0.5200, Loss: 0.6288
Epoch: 006, Train Acc: 0.5133, Test Acc: 0.4800, Loss: 1.2265
Epoch: 007, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 1.0973
Epoch: 008, Train Acc: 0.5467, Test Acc: 0.6000, Loss: 0.6913
Epoch: 009, Train Acc: 0.5067, Test Acc: 0.5400, Loss: 0.8552
Epoch: 010, Train Acc: 0.5400, Test Acc: 0.6400, Loss: 0.8330
Epoch: 011, Train Acc: 0.5000, Test Acc: 0.4800, Loss: 0.6977
Epoch: 012, Train Acc: 0.5267, Test Acc: 0.5800, Loss: 0.6671
Epoch: 013, Train Acc: 0.5800, Test Acc: 0.6400, Loss: 0.6874
Epoch: 014, Train Acc: 0.5667, Test Acc: 0.6800, Loss: 0.6523
Epoch: 015, Train Acc: 0.5267, Test Acc: 0.5800, Loss: 0.6844
Epoch: 016, Train Acc: 0.5200, Test Acc: 0.5800, Loss: 0.7007
Epoch: 

In [77]:
from torch.nn import Linear
from torch_geometric.nn import GINConv, JumpingKnowledge, global_mean_pool
from torch.nn import BatchNorm1d as BN
from torch.nn import Linear, ReLU, Sequential
#from torch_geometric.nn import JumpingKnowledge, SAGEConv, global_mean_pool

class GIN(torch.nn.Module):
    def __init__(self):
        super(GIN, self).__init__()
        self.conv1 = GINConv(
            Sequential(
                Linear(1, 16),
                ReLU(),
                Linear(16, 16),
                ReLU(),
                BN(16),
            ), train_eps=False)
        self.conv2 = GINConv(
            Sequential(
                Linear(16, 32),
                ReLU(),
                Linear(32, 32),
                ReLU(),
                BN(32),
            ), train_eps=False)
        self.conv3 = GINConv(
            Sequential(
                Linear(32, 32),
                ReLU(),
                Linear(32, 32),
                ReLU(),
                BN(32),
            ), train_eps=False)
        self.lin1 = Linear(32, 16)
        self.lin2 = Linear(16, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = self.conv3(x, edge_index)
        x = global_mean_pool(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)

In [78]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GIN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 1000):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.7262
Epoch: 002, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6956
Epoch: 003, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6947
Epoch: 004, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6984
Epoch: 005, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6942
Epoch: 006, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6928
Epoch: 007, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6936
Epoch: 008, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6942
Epoch: 009, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6943
Epoch: 010, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6948
Epoch: 011, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6965
Epoch: 012, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6911
Epoch: 013, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6920
Epoch: 014, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6960
Epoch: 015, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6925
Epoch: 016, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6899
Epoch: 0

In [79]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.data import Batch
from torch_geometric.nn import (
    GraphConv,
    JumpingKnowledge,
    global_mean_pool,
    graclus,
    max_pool,
)

num_layers  = 4
hidden = 16
class Graclus(torch.nn.Module):
    def __init__(self):
        super(Graclus, self).__init__()
        self.conv1 = GraphConv(1, 16, aggr='mean')
        self.convs = torch.nn.ModuleList()
        for i in range(num_layers - 1):
            self.convs.append(GraphConv(16, 16, aggr='mean'))
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(64, 16)
        self.lin2 = Linear(16, 1)

    def forward(self, data):
        x, edge_index, weight , batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                cluster = graclus(edge_index,num_nodes=x.size(0))
                data = Batch(x=x, edge_index=edge_index, batch=batch)
                data = max_pool(cluster, data)
                x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        return x

In [80]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Graclus().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 1000):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.7003
Epoch: 002, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6926
Epoch: 003, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6973
Epoch: 004, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6955
Epoch: 005, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6916
Epoch: 006, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6956
Epoch: 007, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6914
Epoch: 008, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6908
Epoch: 009, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6906
Epoch: 010, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6946
Epoch: 011, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6932
Epoch: 012, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6935
Epoch: 013, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6934
Epoch: 014, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6926
Epoch: 015, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6917
Epoch: 016, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6925
Epoch: 0

In [81]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    GraphConv,
    JumpingKnowledge,
    TopKPooling,
    global_mean_pool,
)


class TopK(torch.nn.Module):
    def __init__(self):
        super(TopK, self).__init__()
        self.ratio = 0.8
        self.num_layers = 3
        self.hidden = 16
        self.conv1 = GraphConv(1, 16, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(16, 16, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend(
            [TopKPooling(16, self.ratio) for i in range((self.num_layers) // 2)])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(48, 16)
        self.lin2 = Linear(16, 1)


    def forward(self, data):
        x, edge_index, weight, batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, _, batch, _, _ = pool(x, edge_index,
                                                     batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        #return F.log_softmax(x, dim=-1)
        return x

In [82]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TopK().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6992
Epoch: 002, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.7008
Epoch: 003, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6956
Epoch: 004, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6885
Epoch: 005, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6873
Epoch: 006, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6914
Epoch: 007, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6979
Epoch: 008, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6940
Epoch: 009, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6954
Epoch: 010, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6965
Epoch: 011, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6944
Epoch: 012, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.7010
Epoch: 013, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6961
Epoch: 014, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6973
Epoch: 015, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6936
Epoch: 016, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6942
Epoch: 0

In [83]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    GraphConv,
    JumpingKnowledge,
    SAGPooling,
    global_mean_pool,
)


class SAGPool(torch.nn.Module):
    def __init__(self):
        super(SAGPool, self).__init__()
        self.num_layers = 4
        self.hidden = 32
        self.ratio = 0.8
        self.conv1 = GraphConv(1, self.hidden, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(self.hidden, self.hidden, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend(
            [SAGPooling(self.hidden, self.ratio) for i in range((self.num_layers) // 2)])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(self.num_layers * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, weight, batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, _, batch, _, _ = pool(x, edge_index,
                                                     batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)

In [84]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SAGPool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.7116
Epoch: 002, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.7073
Epoch: 003, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.7247
Epoch: 004, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6813
Epoch: 005, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6925
Epoch: 006, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6915
Epoch: 007, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6942
Epoch: 008, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6956
Epoch: 009, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6928
Epoch: 010, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6930
Epoch: 011, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6933
Epoch: 012, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6929
Epoch: 013, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6911
Epoch: 014, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.7046
Epoch: 015, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6892
Epoch: 016, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6976
Epoch: 0

In [85]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    EdgePooling,
    GraphConv,
    JumpingKnowledge,
    global_mean_pool,
)


class EdgePool(torch.nn.Module):
    def __init__(self):
        super(EdgePool, self).__init__()
        self.num_layers = 3 
        self.hidden = 16
        self.conv1 = GraphConv(1, self.hidden, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(self.hidden, self.hidden, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend(
            [EdgePooling(self.hidden) for i in range((self.num_layers) // 2)])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(self.num_layers * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, weight ,  batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, batch, _ = pool(x, edge_index, batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [86]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EdgePool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6925
Epoch: 002, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6929
Epoch: 003, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6948
Epoch: 004, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.7015
Epoch: 005, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6907
Epoch: 006, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6929
Epoch: 007, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6926
Epoch: 008, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6928
Epoch: 009, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6923
Epoch: 010, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6935
Epoch: 011, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6960
Epoch: 012, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.7100
Epoch: 013, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6949
Epoch: 014, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6968
Epoch: 015, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6978
Epoch: 016, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6923
Epoch: 0

In [87]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import GlobalAttention, SAGEConv


class GlobalAttentionNet(torch.nn.Module):
    def __init__(self):
        super(GlobalAttentionNet, self).__init__()
        self.num_layers = 3
        self.hidden  = 16
        self.conv1 = SAGEConv(1, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(SAGEConv(self.hidden, self.hidden))
        self.att = GlobalAttention(Linear(self.hidden, 1))
        self.lin1 = Linear(self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = self.att(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)


In [88]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GlobalAttentionNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6943
Epoch: 002, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6890
Epoch: 003, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6976
Epoch: 004, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.7011
Epoch: 005, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6926
Epoch: 006, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6917
Epoch: 007, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6920
Epoch: 008, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6951
Epoch: 009, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6892
Epoch: 010, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6921
Epoch: 011, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6913
Epoch: 012, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6958
Epoch: 013, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6942
Epoch: 014, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6957
Epoch: 015, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6937
Epoch: 016, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6935
Epoch: 0

In [89]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import SAGEConv, Set2Set


class Set2SetNet(torch.nn.Module):
    def __init__(self):
        super(Set2SetNet,self).__init__()
        self.hidden = 16
        self.num_layers = 3 
        self.conv1 = SAGEConv(1, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(SAGEConv(self.hidden, self.hidden))
        self.set2set = Set2Set(self.hidden, processing_steps=4)
        self.lin1 = Linear(2 * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = self.set2set(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [90]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Set2SetNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6906
Epoch: 002, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.7040
Epoch: 003, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6925
Epoch: 004, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6951
Epoch: 005, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6961
Epoch: 006, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6903
Epoch: 007, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6939
Epoch: 008, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6964
Epoch: 009, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.7000
Epoch: 010, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6963
Epoch: 011, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6921
Epoch: 012, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6926
Epoch: 013, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6916
Epoch: 014, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6915
Epoch: 015, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6934
Epoch: 016, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6977
Epoch: 0

In [91]:
import torch
import torch.nn.functional as F
from torch.nn import Conv1d, Linear

from torch_geometric.nn import SAGEConv, global_sort_pool


class SortPool(torch.nn.Module):
    def __init__(self):
        super(SortPool, self).__init__()
        self.k = 30
        self.num_layers = 3 
        self.hidden = 16
        self.conv1 = SAGEConv(1, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(SAGEConv(self.hidden, self.hidden))
        self.conv1d = Conv1d(self.hidden, 32, 5)
        self.lin1 = Linear(32 * (self.k - 5 + 1), self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = global_sort_pool(x, batch, self.k)
        x = x.view(len(x), self.k, -1).permute(0, 2, 1)
        x = F.relu(self.conv1d(x))
        x = x.view(len(x), -1)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [92]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SortPool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6975
Epoch: 002, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6961
Epoch: 003, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6897
Epoch: 004, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6928
Epoch: 005, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6930
Epoch: 006, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6942
Epoch: 007, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6929
Epoch: 008, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6924
Epoch: 009, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6918
Epoch: 010, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6913
Epoch: 011, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6916
Epoch: 012, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6905
Epoch: 013, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6925
Epoch: 014, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.7001
Epoch: 015, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6940
Epoch: 016, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6911
Epoch: 0

In [93]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    ASAPooling,
    GraphConv,
    JumpingKnowledge,
    global_mean_pool,
)


class ASAP(torch.nn.Module):
    def __init__(self):
        super(ASAP, self).__init__()
        self.num_layers = 5 
        self.hidden  = 32
        self.ratio = 0.8
        self.dropout = 0.2
        self.conv1 = GraphConv(1, self.hidden, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(self.hidden, self.hidden, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend([
            ASAPooling(self.hidden, self.ratio, dropout=self.dropout)
            for i in range((self.num_layers) // 2)
        ])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(self.num_layers * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, edge_weight, batch = data.x, data.edge_index,data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = conv(x=x, edge_index=edge_index, edge_weight=edge_weight)
            x = F.relu(x)
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, edge_weight, batch, _ = pool(
                    x=x, edge_index=edge_index, edge_weight=edge_weight,
                    batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)


In [94]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ASAP().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6937
Epoch: 002, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6892
Epoch: 003, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6905
Epoch: 004, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6928
Epoch: 005, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6925
Epoch: 006, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6909
Epoch: 007, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6980
Epoch: 008, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6927
Epoch: 009, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6900
Epoch: 010, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6909
Epoch: 011, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6929
Epoch: 012, Train Acc: 0.5067, Test Acc: 0.5000, Loss: 0.6929
Epoch: 013, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6951
Epoch: 014, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6932
Epoch: 015, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6962
Epoch: 016, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6958
Epoch: 0